In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.metrics import roc_auc_score


In [2]:
train = pd.read_pickle("data/train.pkl").dropna()
test = pd.read_pickle("data/test.pkl").dropna()
y_train = np.array(train['label'])
y_test = np.array(test['label'])

In [3]:
train_parent_tdidf_csr = vstack(train["parent_comment_tdidf"])
test_parent_tdidf_csr = vstack(test["parent_comment_tdidf"])

train_tdidf_csr = vstack(train["comment_tdidf_nn"])
test_tdidf_csr = vstack(test["comment_tdidf_nn"])

train_parent_bow_csr = vstack(train["parent_comment_bow"])
test_parent_bow_csr = vstack(test["parent_comment_bow"])

train_bow_csr = vstack(train["comment_bow"])
test_bow_csr = vstack(test["comment_bow"])

In [10]:
train_tdidf_csr

<161010x8160 sparse matrix of type '<class 'numpy.float64'>'
	with 855934 stored elements in Compressed Sparse Row format>

In [4]:
list_of_features = [
       'comment_word_count','parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count',
       'weighted_parent_sentiment_score_neutral',
       'weighted_parent_sentiment_score_positive',
       'weighted_comment_sentiment_score_neutral',
       'weighted_comment_sentiment_score_positive']
bool_cols = ['weighted_parent_sentiment_score_neutral',
             'weighted_parent_sentiment_score_positive',
             'weighted_comment_sentiment_score_neutral',
             'weighted_comment_sentiment_score_positive']

for col in bool_cols: #need to convert bool type to integer
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)
X_train_gen_features = csr_matrix(train[list_of_features])

X_test_gen_features = csr_matrix(test[list_of_features])
x_train = hstack([X_train_gen_features,train_tdidf_csr]).toarray() #deep learning automates feature selection. from our supervised learning we have learnt that BoW adds no information given tf-idf.

In [9]:
#x_train = train_tdidf_csr.toarray()
x_test = hstack([X_test_gen_features,test_tdidf_csr]).toarray()

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, epochs=10, batch_size=8, validation_split=0.1)


Epoch 1/10
18114/18114 [==============================] - 40s 2ms/step - loss: 0.6732 - accuracy: 0.5760 - val_loss: 0.7190 - val_accuracy: 0.5903
Epoch 2/10
18114/18114 [==============================] - 38s 2ms/step - loss: 0.6242 - accuracy: 0.6512 - val_loss: 0.6136 - val_accuracy: 0.6599
Epoch 3/10
18114/18114 [==============================] - 37s 2ms/step - loss: 0.6101 - accuracy: 0.6673 - val_loss: 0.6169 - val_accuracy: 0.6546
Epoch 4/10
18114/18114 [==============================] - 36s 2ms/step - loss: 0.5974 - accuracy: 0.6784 - val_loss: 0.6183 - val_accuracy: 0.6500
Epoch 5/10
18114/18114 [==============================] - 37s 2ms/step - loss: 0.5907 - accuracy: 0.6854 - val_loss: 0.5950 - val_accuracy: 0.6765
Epoch 6/10
18114/18114 [==============================] - 37s 2ms/step - loss: 0.5854 - accuracy: 0.6897 - val_loss: 0.6239 - val_accuracy: 0.6465
Epoch 7/10
18114/18114 [==============================] - 36s 2ms/step - loss: 0.5820 - accuracy: 0.6944 - val_loss: 0

In [10]:
y_pred_test = model.predict(x_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)
print(f'AUC: {roc_auc_test}')
#comment td-idf only AUC: 0.6628971516171749
#gen features + comment td-idf AUC: 0.7419122793471202

1259/1259 [==============================] - 1s 864us/step
AUC: 0.7419122793471202


In [16]:
yucai = test[["comment"]]
yucai_ans = test[["label"]]
